In [13]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['페르소나 채팅']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)

#### prepare for task preprocess
tools = [
    {
        'name': 'update_persona',
        'description': 'Update personas discovered in conversations',
        'parameters': {
            'type': 'object',
            'properties': {
                'persona': {
                    'type': 'string',
                    'description': 'persona to update'},
            },
            'required': ['persona']
        }
    },
    {
        'name': 'get_persona_summary',
        'description': 'Get summary of personas discovered in conversations',
        'parameters': {
            'type': 'object',
            'properties': {},
            'required': []
        }
    },
]
function_call = {
    'update_persona':{'name': 'update_persona', 'arguments': {'persona': ''}},
    'get_persona_summary': {'name': 'get_persona_summary', 'arguments': {}},
}


#### prepare for task preprocess end
from collections import Counter
session_restart_counter = Counter()

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            sessions = line['sessionInfo']
            start_last_utt = [(sess['dialog'][0], sess['dialog'][-1]) for sess in sessions]
            start_last_utt = [(start['speaker'], last['speaker']) for start, last in start_last_utt]
            for i in range(len(start_last_utt)-1):
                session_restart_counter[(start_last_utt[0][0], start_last_utt[i][1], start_last_utt[i+1][0])] += 1
            #### data preprocess end 

            #### 페르소나 채팅
            data = {'input': None, 'output': None}
            ## preprocess data from line

            ## preprocess data from line end
            # task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 페르소나 채팅 end
            
            # break
        # break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 3/3 [00:00<00:00,  3.60it/s]
